<a href="https://colab.research.google.com/github/felipeProgrammingDesign/BuscaSequencial/blob/master/C%C3%B3pia_de_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/andrewyzy/pnfiles/raw/master/dataset.tar.gz

!tar -zxvf dataset.tar.gz

A saída de streaming foi truncada nas últimas 5000 linhas.
dataset/test/4755.png
dataset/test/4098.png
dataset/test/475.png
dataset/test/5202.png
dataset/test/4765.png
dataset/test/4282.png
dataset/test/4670.png
dataset/test/2566.png
dataset/test/163.png
dataset/test/1549.png
dataset/test/5333.png
dataset/test/1048.png
dataset/test/2670.png
dataset/test/5021.png
dataset/test/3144.png
dataset/test/110.png
dataset/test/1415.png
dataset/test/4987.png
dataset/test/4173.png
dataset/test/2734.png
dataset/test/1411.png
dataset/test/798.png
dataset/test/4800.png
dataset/test/2712.png
dataset/test/2213.png
dataset/test/4969.png
dataset/test/967.png
dataset/test/1369.png
dataset/test/5106.png
dataset/test/2575.png
dataset/test/3735.png
dataset/test/586.png
dataset/test/3609.png
dataset/test/801.png
dataset/test/4060.png
dataset/test/3065.png
dataset/test/2444.png
dataset/test/1849.png
dataset/test/146.png
dataset/test/5419.png
dataset/test/3066.png
dataset/test/1089.png
dataset/test/4470.png
dat

Mounted at /content/drive


Aqui nós vamos carregar as imagens, labels, separar os dados e fazer o sorteio.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import numpy as np
import cv2
import os

def load_multiclass_dataset(path, height=224, width=224, num_channels=1):
	classes = sorted(os.listdir(path))
	images = [sorted(os.listdir(path+'/'+id)) for id in classes]
	num_images = np.sum([len(l) for l in images])

	X = np.empty([num_images, height, width, num_channels], dtype=np.uint8)
	y = np.empty([num_images], dtype=np.int64)

	k = 0
	for i in range(len(classes)):
		for j in range(len(images[i])):
			img = cv2.imread(path+'/'+classes[i]+'/'+images[i][j], cv2.IMREAD_GRAYSCALE).reshape(height, width, num_channels)
			assert img.shape == (height, width, num_channels), "%r has an invalid image size!" % images[i][j]
			assert img.dtype == np.uint8, "%r has an invalid pixel format!" % images[i][j]
			X[k] = img
			y[k] = i
			k += 1

	return X, y, classes


def shuffle(X, y, seed=None):
	assert len(X) == len(y), "The 1st dimension size must be the same for both arrays!"
	if seed is not None:
		np.random.seed(seed)
	p = np.random.permutation(len(X))
	return X[p], y[p]

def split(X, y, rate):
	assert len(X) == len(y), "The 1st dimension size must be the same for both arrays!"
	idx = int(len(X)*float(rate))
	return X[:idx], y[:idx], X[idx:], y[idx:]


def randomize(a, b):
    permutation = np.random.permutation(a.shape[0])
    shuffled_a = a[permutation]
    shuffled_b = b[permutation]
    return shuffled_a, shuffled_b

def shuffle_split_data(X, y, proporcao):
    np.random.seed(50)
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, proporcao)
    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]
    return X_train, y_train, X_test, y_test


In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()   
import numpy as np
import random
import time
import cv2
import os, glob
import sys
import os
from os import listdir
import re

TRAIN_FOLDER = './dataset/train'
TEST_FOLDER = './dataset/test'  
SPLIT_RATE = 0.8   

IMAGE_HEIGHT = 71
IMAGE_WIDTH = 77  
NUM_CHANNELS = 1  


def read_images_test(dataset_path):
    imagepaths, labels = list(), list()
    images_list = []
    for i in range(1, 5491):
        labels.append(str(i)+".png")
        img = cv2.imread("./dataset/test/"+str(i)+".png", cv2.COLOR_BGR2GRAY)

        reshaped = np.reshape(img,  (img.shape[0], -1) )

        reshaped = reshaped.flatten()
        images_list.append(reshaped)

    images_tensor = np.array(images_list, dtype=np.uint8)
    print(images_tensor.shape)
    return images_tensor, labels

test, labels_test = read_images_test(TEST_FOLDER)

test = test/255.0
X_test = test


X_train, y_train, classes_train = load_multiclass_dataset(TRAIN_FOLDER, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)
X_train = X_train.reshape(-1, IMAGE_HEIGHT*IMAGE_WIDTH*NUM_CHANNELS)/255.


X_train, y_train = shuffle(X_train, y_train, seed=42)
X_train, y_train, X_val, y_val = split(X_train, y_train, SPLIT_RATE)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)


graph = tf.Graph()

with graph.as_default():

	X = tf.placeholder(tf.float32, shape = (None, IMAGE_HEIGHT*IMAGE_WIDTH*NUM_CHANNELS))
	y = tf.placeholder(tf.int64, shape = (None,))
	y_one_hot = tf.one_hot(y, len(classes_train))
	learning_rate = tf.placeholder(tf.float32)

	out = tf.layers.dense(X, len(classes_train), activation=tf.nn.relu)
 
	loss = tf.reduce_mean(tf.reduce_sum((y_one_hot-out)**2))

	train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

	result = tf.argmax(out, 1)
	correct = tf.reduce_sum(tf.cast(tf.equal(result, y), tf.float32))


def training_epoch(session, op, lr):
	batch_list = np.random.permutation(len(X_train))

	start = time.time()
	train_loss = 0
	train_acc = 0
	for j in range(0, len(X_train), BATCH_SIZE):
		if j+BATCH_SIZE > len(X_train):
			break
		X_batch = X_train.take(batch_list[j:j+BATCH_SIZE], axis=0)
		y_batch = y_train.take(batch_list[j:j+BATCH_SIZE], axis=0)

		ret = session.run([op, loss, correct], feed_dict = {X: X_batch, y: y_batch, learning_rate: lr})
		train_loss += ret[1]*BATCH_SIZE
		train_acc += ret[2]

	pass_size = (len(X_train)-len(X_train)%BATCH_SIZE)
	print('Training Epoch:'+str(epoch)+' LR:'+str(lr)+' Time:'+str(time.time()-start)+' ACC:'+str(train_acc/pass_size)+' Loss:'+str(train_loss/pass_size))
	return train_loss/pass_size


def evaluation(session, Xv, yv, name='Evaluation'):
	start = time.time()
	eval_loss = 0
	eval_acc = 0
	for j in range(0, len(Xv), BATCH_SIZE):
		ret = session.run([loss, correct], feed_dict = {X: Xv[j:j+BATCH_SIZE], y: yv[j:j+BATCH_SIZE]})
		eval_loss += ret[0]*min(BATCH_SIZE, len(Xv)-j)
		eval_acc += ret[1]

	tf.summary.histogram("Loss", eval_loss/len(Xv))
	print(name+' Epoch:'+str(epoch)+' Time:'+str(time.time()-start)+' ACC:'+str(eval_acc/len(Xv))+' Loss:'+str(eval_loss/len(Xv)))

	return eval_acc/len(Xv), eval_loss/len(Xv)


NUM_EPOCHS_FULL = 5
S_LEARNING_RATE_FULL = 1
BATCH_SIZE = 2


with tf.Session(graph = graph) as session:
	session.run(tf.global_variables_initializer())

	loss_training = []
	loss_validation = []

	for epoch in range(NUM_EPOCHS_FULL):
		lr = S_LEARNING_RATE_FULL
		loss_return = training_epoch(session, train_op, lr)
		loss_training.append(loss_return)
	

	
	file = open("FELIPE_AUGUSTO.txt","w") 
	string = ""
	result = tf.argmax(out, 1)
	for i in range(len(X_test)):
		saida = result.eval(feed_dict={X: [X_test[i]]}, session=session)

		a = str(labels_test[i])+" "+str(saida)+"\n"
		a = a.replace('[', '')
		a = a.replace(']', '')
	
		string = string + a

	file.write(string)
	file.close() 

	




(5490, 5467)
(4000, 5467) (4000,) (1000, 5467) (1000,)


/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Training Epoch:0 LR:1 Time:2.682363748550415 ACC:0.1035 Loss:2.0002547796964647
Training Epoch:1 LR:1 Time:2.6045069694519043 ACC:0.10375 Loss:2.0
Training Epoch:2 LR:1 Time:2.678044319152832 ACC:0.10375 Loss:2.0
Training Epoch:3 LR:1 Time:2.6053714752197266 ACC:0.10375 Loss:2.0
Training Epoch:4 LR:1 Time:2.7377538681030273 ACC:0.10375 Loss:2.0
